In [ ]:
#統整

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, IDF, CountVectorizer
from pyspark.sql.functions import rand 
from pyspark.sql.functions import lit
import pyspark.sql.functions as sparkf
import pyspark.sql.functions as F
from pyspark.sql import Window

'''
if __name__ == "__main__":

    spark = SparkSession \
        .builder \
        .getOrCreate()     
''' '''
data_gnd 答案 joined_bott_top_aswr tag=1
rand_gnd 問題 joined_bott_top_ques tag=0 '''

#Aset
data_gnd_pre = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/gnd_top_bottom_pairs.csv", inferSchema=True, header=False)
data_gnd = data_gnd_pre.na.drop()

joined_bott = data_gnd.join(ready_data_bott, data_gnd._c1 == ready_data_bott.bot_id, 'inner').select('*')
joined_bott_top_aswr = joined_bott.join(ready_data_top, joined_bott._c0 == ready_data_top.top_id, 'inner' ).select('bot_id', 'tf_idf_b', 'tf_idf_c', 'top_id', 'tf_idf_b_top', 'tf_idf_c_top')
joined_bott_top_aswr = joined_bott_top_aswr.withColumn('tag', lit(1))#.show(10) #.count() #20655

#Qset
w = Window().orderBy(F.lit("rand"))
bott_id_rand = data_gnd.select('_c0').withColumn('rand', F.row_number().over(w)).orderBy('rand')
top_id_rand = data_gnd.select('_c1').withColumn('pre_rand', sparkf.monotonically_increasing_id()).orderBy(rand())
top_id_rand_2 = top_id_rand.select('_c1').withColumn('rand', F.row_number().over(w)).orderBy('rand')
rand_gnd = bott_id_rand.join(top_id_rand_2, bott_id_rand.rand == top_id_rand_2.rand ).select('_c0', '_c1')

joined_bott2 = rand_gnd.join(ready_data_bott, data_gnd._c1 == ready_data_bott.bot_id).select('*')
joined_bott_top_ques = joined_bott2.join(ready_data_top, joined_bott._c0 == ready_data_top.top_id, 'inner' ).select('bot_id', 'tf_idf_b', 'tf_idf_c', 'top_id', 'tf_idf_b_top', 'tf_idf_c_top')
joined_bott_top_ques = joined_bott_top_ques.withColumn('tag', lit(0))#.show(10)  #.count() #20655

#Q&A set
pre_dl_dataset = joined_bott_top_ques.union(joined_bott_top_aswr)
pre_dl_dataset.show()
pre_dl_dataset.count() #41310

#存起來

pandasDF2 = pre_dl_dataset.toPandas()
pandasDF2.to_csv("./pre_dl_dataset.csv") 
#pandasDF2.head(10) #.count()

# +---------+--------------------+--------------------+---------+--------------------+--------------------+---+
# |   bot_id|            tf_idf_b|            tf_idf_c|   top_id|        tf_idf_b_top|        tf_idf_c_top|tag|
# +---------+--------------------+--------------------+---------+--------------------+--------------------+---+
# |191564594|(6437,[2,4,53,164...|(1738,[0,1,5,12,5...| 47549965|(7041,[296,1590,3...|(1859,[0,1,3,5,7]...|  0|
# |195060772|(6437,[2,11,36,37...|(1738,[0,1,5,22,3...| 95985283|(7041,[4,165,880,...|(1859,[0,1,2,4,29...|  0|
# |190366907|(6437,[0,39,62,63...|(1738,[0,1,2,21,2...| 96517114|(7041,[15,64,65,7...|(1859,[0,1,2,4,18...|  0|
# |157180593|(6437,[0,133,170]...|(1738,[0,1,2,7,8,...|100745635|(7041,[30,48,190,...|(1859,[0,1,2,13,1...|  0|



In [ ]:
# Prepare data -Preview
#bott
data_bott = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/bottom.csv", inferSchema=True, header=True)

data_bott.show(100,truncate=False)
data_bott.distinct()
data_bott.distinct().count() #13662

#top
data_top = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/top.csv", inferSchema=True, header=True)

data_top.show(100, False)
data_top.distinct()
data_top.distinct().count() #14871

# .....nlp_feature_engineering

In [ ]:
#ver2 addition

#make tag

#title split with - into 2 and create columns    as  brand & product desc
words_added = Tokenizer(inputCol="message", outputCol="words").transform(df)
words_added.show(truncate=False)
#category split with > into 4 and create columns  
#ttl 7 col

header= ["PID","Brand", "Desc", "Cate1", "Cate2", "Cate3", "Cate4"] 

-----------------------------------------------------
id_1 | id_2 | Desc(tf-idf) | cate1(string[1.2.3.4...]]) | cate2(string) | cate3(string) | cate4(string)  | ..id2's'......... | y = [1, 0, 1, 0,0,0,0,0...]
------------------------------------------------------

#nlp set
# 1 + 2 > 3  = current hot  X
# 1 + 2 > 4  = core product X
# 3 + 4+5+6+7  > 1
# 3 + 4/5/6/7  > 1    100D:based on gnd
